In [ ]:
import random
import json
import pickle
import numpy as np

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open("/content/intents.json").read())

In [ ]:
words = []
classes = []
documents = []
ignore_words = ['!','.','?',',']  #ignored words

In [ ]:
nltk.download('punkt_tab')
nltk.download('wordnet')

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)  #splitting sentence to list of words
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_words]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words,open('/content/words.pkl','wb'))
pickle.dump(classes,open('/content/classes.pkl','wb'))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
training = []
output_empty = [0]*len(classes)

In [ ]:
for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (111, 2) + inhomogeneous part.

In [ ]:
len(training)

111

In [ ]:
max_len = max(len(x[0]) for x in training)
max_len

142

In [ ]:
for i in range(len(training)):
    bag_len = len(training[i][1])
    if bag_len < max_len:
        padding = [0] * (max_len - bag_len)
        training[i][1].extend(padding)

In [ ]:
training = np.array(training)

In [ ]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save(chatbot_loki.model)

Epoch 1/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0139 - loss: 4.9518
Epoch 2/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0803 - loss: 4.8468
Epoch 3/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0890 - loss: 4.6436
Epoch 4/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1364 - loss: 4.2716
Epoch 5/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0967 - loss: 4.0127
Epoch 6/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1560 - loss: 3.7402
Epoch 7/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1084 - loss: 3.5312
Epoch 8/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1417 - loss: 3.2604
Epoch 9/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1340 - loss: 3.2828
Epoch 10/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1582 - loss: 3.2437
Epoch 11/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1849 - loss: 2.7717
Epoch 12/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accura

NameError: name 'chatbot_loki' is not defined

In [ ]:
model.save('chatbot_loki.h5')

In [ ]:
model.save('chatbot_loki.keras')

In [ ]:
from tensorflow.keras.models import load_model
model_loaded = load_model('/content/chatbot_loki.h5')

In [ ]:
model_loaded.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │          18,304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 142)                 │           9,230 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 35,792 (139.82 KB)

 Trainable params: 35,790 (139.80 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [42]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [43]:
def clean_up(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [44]:
def bag_of_words(sentence):
    sentence_words = clean_up(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i,word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)


In [45]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model_loaded.predict(np.array([bow]))[0]
    Error_threshold =0.25
    results = [[i,r] for  i,r in enumerate(res) if r > Error_threshold]

    results.sort(key=lambda x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability': str(r[1])})
    return return_list

In [46]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [48]:
while True:
    message = input("You: ")
    if message.lower() in ["stop", "exit", "quit"]:
        print("Chatbot stopped.")
        break
    ints = predict_class(message)
    res = get_response(ints, intents)
    print("Bot:", res)

You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Bot: Hi there, how can I help?
You: who are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Bot: I am Loki, a Deep-Learning chatbot
You: tell me joke
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Bot: How does a penguin build a house? Igloos it together
You: nice
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Bot: Hi there, how can I help?
You: tell me a riddle
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Bot: How do you make the number one disappear?.....Add the letter G and it’s 'gone'!
You: who is pramod
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Bot: Head over to his any of his social profiles to find out! Linkedin: www.linkedin.com/in/kbpramod Github: https://github.com/kbpramod
You: thank you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Bot: My pleasure
You: bye
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Bot: Hi there, how can I help?
You: exit
Chatbot stopped.
